In [1]:
from qiskit_ibm_runtime.fake_provider import FakeKyiv, FakeKyoto
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT
from qiskit.circuit import Instruction, Parameter

from colorama import Fore


from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from utils import check_connections_backend, check_connections_circuit

In [2]:
backend_kyiv = FakeKyiv()
backend_kyoto= FakeKyoto()

In [3]:
qc = QFT(9)
pm_kyiv = generate_preset_pass_manager(optimization_level=1, backend=backend_kyiv)
circuit_isa_kyiv = pm_kyiv.run(qc)

pm_kyoto = generate_preset_pass_manager(optimization_level=1, backend=backend_kyoto)
circuit_isa_kyoto = pm_kyoto.run(qc)



In [4]:
grader_operation_check_message = 'Basis gate check passed!'
def grader_operation_check(backend, circuit):
    op_names_backend = backend.operation_names
    op_names_circuit_dict = circuit.count_ops().items()

    for keys, val in op_names_circuit_dict:
        if keys in op_names_backend:
            None
        else : 
            print(Fore.RED + f'The backend does not support {keys} gate')

    return print(Fore.GREEN + grader_operation_check_message)

In [5]:
grader_operation_check(backend_kyiv, circuit_isa_kyiv)


Basis gate check passed!


In [6]:
grader_operation_check(backend_kyiv, circuit_isa_kyoto)


Basis gate check passed!


In [7]:
grader_connection_check_correct_message = 'The connections match with the backend'
grader_connection_check_incorrect_message = 'The connections does not match with the backend'

def grader_connection_check(backend, circuit):
    conn_backend = check_connections_backend(backend)
    conn_circ = check_connections_circuit(circuit)

    for i in range(len(conn_backend)):
        conn_backend_rz = set(conn_backend[0])
        conn_backend_ecr = set(conn_backend[1])
        conn_backend_sx = set(conn_backend[2])
        conn_backend_x = set(conn_backend[3])

        conn_circuit_rz = set(conn_circ[0])
        conn_circuit_ecr = set(conn_circ[1])
        conn_circuit_sx = set(conn_circ[2])
        conn_circuit_x = set(conn_circ[3])

    if (conn_circuit_rz.issubset(conn_backend_rz) and conn_circuit_rz.issubset(conn_backend_rz) 
        and conn_circuit_ecr.issubset(conn_backend_ecr) and conn_circuit_rz.issubset(conn_backend_rz)):
        print(Fore.GREEN + grader_connection_check_correct_message)

    else:
        print(Fore.RED + grader_connection_check_incorrect_message)





    

In [8]:
grader_connection_check(backend_kyiv, circuit_isa_kyiv)


The connections match with the backend


In [9]:
grader_connection_check(backend_kyiv, circuit_isa_kyoto)


The connections does not match with the backend
